In [7]:
import os
import numpy as np
import cv2
import json

In [8]:
# Create folder to save all data
training_data_path = '/Users/am/Desktop/HKU/InnoWing/project/CNN/data/training-data'
validation_data_path = '/Users/am/Desktop/HKU/InnoWing/project/CNN/data/validation-data'

if not os.path.exists(f'{training_data_path}/img'):
    os.makedirs(f'{training_data_path}/img')

if not os.path.exists(f'{validation_data_path}/img'):
    os.makedirs(f'{validation_data_path}/img')

In [9]:
# Generate images for training
def generate_images(data_size, folder_path, max_blobs=10):
    # Create COCO format dictionary
    annotations = { "images": [], "annotations": [], "categories": [{"id": 1, "name": "human"}] }

    for i in range(data_size):
        img = np.zeros((24, 32), dtype=np.float32)
        num_of_human = np.random.randint(0, max_blobs+1)
        
        for _ in range(num_of_human):

            # Randomly generate a blob
            blob_size = np.random.randint(3, 7)
            x1, y1 = np.random.randint(0, 32-blob_size), np.random.randint(0, 24-blob_size)
            x2, y2 = x1+blob_size, y1+blob_size
            img[y1:y2, x1:x2] = np.random.uniform(28, 35, (blob_size, blob_size))

            # Bounding box
            bbox = [x1, y1, blob_size, blob_size]   # x, y, w, h

            # Mask
            mask = np.zeros((24, 32), dtype=np.uint8)
            mask[y1:y2, x1:x2] = 1

            # Add annotation
            annotations["annotations"].append({
                "id": len(annotations["annotations"])+1,
                "image_id": i+1,
                "category_id": 1,
                "segmentation": [mask.tolist()],
                "area": blob_size*blob_size,
                "bbox": bbox,
                "iscrowd": 0,
                "ignore": 0
            })

        # Add background temperature
        background_temp = np.random.uniform(23, 28, (24, 32))
        img[img == 0] = background_temp[img == 0]

        # Add noise
        img += np.random.normal(0, 0.4, (24, 32))

        # Normalize image
        img = (img - 23) / (35 - 23)
        img_8u = (img * 255).astype(np.uint8)

        # Save image (scaled to 255)
        cv2.imwrite(f'{folder_path}/img/{i}.png'.format(i), img_8u)

        # Add image to COCO format dictionary
        annotations["images"].append({
            "id": i+1,
            "file_name": f"{i}.png".format(i),
            "height": 24,
            "width": 32
    })
        
    return annotations


In [10]:
max_blobs = 10
annotations_training_set = generate_images(500, training_data_path, max_blobs)

# Save annotations to JSON file
with open(f'{training_data_path}/annotations.json', 'w') as f:
    json.dump(annotations_training_set, f)


In [11]:
annotations_validation_set = generate_images(200, validation_data_path, max_blobs)

# Save annotations to JSON file
with open(f'{validation_data_path}/annotations.json', 'w') as f:
    json.dump(annotations_validation_set, f)